# Multi-Agents in GenAI

#### Create a virtual env:
conda create -n langchain_env python=3.11 anaconda
#### Activate the virtual env:
conda activate langchain_env


In [1]:
# install the necessary libraries
!pip install crewai
!pip install crewai_tools
!pip installload_dotenv
!pip install langchain-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.1 MB/s eta 0:00:00
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.5 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 1.7 MB/s eta 0:00:00a 0:00:01
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [39]:
import requests
import os
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from crewai import Agent, Task, Crew
from langchain.tools import tool
from langchain_community.chat_models import ChatOllama

from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
# We will be using Ollama with Llama3 as LLM
# ollama run llama3 # run the line to install ollama 


In [4]:
# Crew Core Concepts
# - Agents
# - Tasks
# - Tools
# - Processes



##Agents:

An agent is an autonomous unit programmed to:

Perform tasks
Make decisions
Communicate with other agents

Think of an agent as a member of a team, with specific skills and a particular job to do. Agents can have different roles like 'Researcher', 'Writer', or 'Customer Support', each contributing to the overall goal of the crew.

Agent Attributes¶

Role	role	Defines the agent's function within the crew. It determines the kind of tasks the agent is best suited for.

Goal	goal	The individual objective that the agent aims to achieve. It guides the agent's decision-making process.

Backstory	backstory	Provides context to the agent's role and goal, enriching the interaction and collaboration dynamics.

LLM (optional)	llm	Represents the language model that will run the agent.  

agent = Agent(
    role="bio agent",

    goal="summarize the short bio for {input} and if needed do more research",

    backstory="agent backstory",

    llm=llm,

    verbose=True,
    
)

load_dotenv()

In [35]:
from crewai import Crew, Agent, Task, Process
from langchain_community.tools import DuckDuckGoSearchRun
from crewai_tools import tool

In [38]:
llm=ChatOllama(model='llama3')

In [40]:

@tool('DuckDuckGoSearch')
def search(search_query: str):
    """Search the web for information on a given topic"""
    return DuckDuckGoSearchRun().run(search_query)

In [41]:
# Define agents with specific roles and tools
researcher = Agent(
    role='Senior Research Analyst',
    goal='Discover innovative AI technologies',
    backstory="""You're a senior research analyst at a large company.
        You're responsible for analyzing data and providing insights
        to the business.
        You're currently working on a project to analyze the
        trends and innovations in the space of artificial intelligence.""",
    tools=[search],
    llm=llm
)

writer = Agent(
    role='Content Writer',
    goal='Write engaging articles on AI discoveries',
    backstory="""You're a senior writer at a large company.
        You're responsible for creating content to the business.
        You're currently working on a project to write about trends
        and innovations in the space of AI for your next meeting.""",
    llm=llm,
    verbose=True
)

In [42]:
# Create tasks for the agents
research_task = Task(
    description='Identify breakthrough AI technologies',
    agent=researcher,
    expected_output='A bullet list summary of the top 5 most important AI news'
)
write_article_task = Task(
    description='Draft an article on the latest AI technologies',
    agent=writer,
    expected_output='3 paragraph blog post on the latest AI technologies'
)

In [45]:
my_crew = Crew(
    agents=[researcher, writer],
    tasks=[research_task, write_article_task],
    process=Process.sequential,
    full_output=True,
    verbose=True,
)

2024-07-23 17:49:30,108 - 8482638848 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


In [47]:
inputs = {'topic': 'AI in healthcare'}
async_result = my_crew.kickoff(inputs=inputs)
print(async_result)

 [2024-07-23 17:49:51][DEBUG]: == Working Agent: Senior Research Analyst
 [2024-07-23 17:49:51][INFO]: == Starting Task: Identify breakthrough AI technologies
 

Artificial intelligence. AI for everything: 10 Breakthrough Technologies 2024. Generative AI tools like ChatGPT reached mass adoption in record time, and reset the course of an entire industry. By . 10 Breakthrough Technologies. Every year, the reporters and editors at MIT Technology Review survey the tech landscape and pick 10 technologies that we think have the greatest potential to change ... In 2024, generative AI might actually become useful for the regular, non-tech person, and we are going to see more people tinkering with a million little AI models. State-of-the-art AI models ... 2023 has unfolded as a testament to the extraordinary advancements in artificial intelligence (AI). In a year awash with groundbreaking technological leaps and profound ethical debates, we have ... The 2024 list of 10 Breakthrough Technologies

KeyboardInterrupt: 